In [1]:
import polars as pl
import pandas as pd
import sys
from tqdm import tqdm
sys.path.append("../src")
import re
from gutatlas.data import filter_by_tag
from gutatlas.utils.constants import MENTAL_HEALTH_TAGS, GI_TAGS
from gutatlas.data import map_gi_status_binary, normalize_multilabel_gi_tags
from gutatlas.features import clean_feature_names, dead_features
import janitor

## Microbiomap

In [ ]:
taxon_path = "../data/microbiomap/raw/taxonomic_table.csv"
metadata_path = "../data/microbiomap/raw/sample_metadata.tsv"
tags_path = "../data/microbiomap/raw/tags.tsv"

sample_metadata = pl.read_csv(metadata_path, separator="\t").with_columns(
    (pl.col("project") + "_" + pl.col("srr")).alias("sample")
)
sample_tags = pl.read_csv(tags_path, separator="\t").with_columns(
    (pl.col("project") + "_" + pl.col("srr")).alias("sample")
)

#taxon table is huge. open batches of 1000, normalize the abundances of each taxon on a per-sample basis
#then save each batch for easier processing going forward
reader = pl.read_csv_batched(taxon_path, batch_size=1000)
first_batch = True
i = 0
while True:
    batches = reader.next_batches(1)
    if not batches:
        break
    batch = batches[0]
    taxon_cols = batch.columns[2:]

#total sum scaling
    batch = batch.with_columns(
        pl.sum_horizontal([pl.col(col) for col in taxon_cols]).alias("total_reads")
    ).with_columns([
        (pl.col(col) / pl.col("total_reads")).alias(col) for col in taxon_cols
    ]).select(["sample"] + taxon_cols)

    merged = (
        batch.join(sample_metadata, on="sample", how="inner")
             .join(sample_tags, on="sample", how="left")
             .drop(["project_right", "srr_right", "srs_right", "total_bases", 
                    "instrument",'srs', 'project', 'srr', 'library_strategy', 'library_source'])
    )

    merged.write_parquet(f"../data/interim/batches/taxa_merged_batch_{i}.parquet")
    i += 1


#lazily read all batches, split by region, and save each region separately
batches = pl.scan_parquet('../data/interim/batches/taxa_merged_batch_*.parquet')

unique_regions = (
    batches.select("iso")
           .unique()
           .collect()
)

for region in unique_regions['iso']:
    split = batches.filter(pl.col('iso') == region).collect()

    split.write_parquet(f'../data/interim/regional_data/{region}_microbiome.parquet')

### GI tags

In [46]:
gi_merged = filter_by_tag('../data/interim/regional_data/',GI_TAGS)
gi_merged.write_parquet('../data/interim/filtered_and_merged/gi_microbiomes_merged.parquet')

#### binary classification set

In [6]:
#remove duplicate rows for each sample, and dead featuyres. only need to know if any disease is present or not 
merged_gi = pd.read_parquet('../data/interim/filtered_and_merged/gi_microbiomes_merged.parquet')
drop_cols = dead_features(merged_gi)

merged_gi['disease_present'] = merged_gi.value.apply(map_gi_status_binary)

gi_training = (merged_gi
                         .sort_values(by = ['disease_present','sample'],ascending=False)
                         .drop_duplicates(subset = 'sample',keep='first')
                         .reset_index(drop=True)
                         .drop(columns = ['pubdate','geo_loc_name','iso','region','tag','value']+drop_cols)
                         .rename(columns = {col:clean_feature_names(col) for col in merged_gi.columns})
                         .clean_names()
                         )


gi_training.to_parquet('../data/processed/gi_binary_training.parquet')

#### multilabel classification set

In [41]:
merged_gi = pd.read_parquet('../data/interim/filtered_and_merged/gi_microbiomes_merged.parquet')

In [43]:
merged_gi.drop_duplicates('sample')

,sample,Bacteria.Actinomycetota.Coriobacteriia.Coriobacteriales.Atopobiaceae.Tractidigestivibacter,Bacteria.Actinomycetota.Coriobacteriia.Coriobacteriales.Coriobacteriaceae.Collinsella,Bacteria.Actinomycetota.Coriobacteriia.Coriobacteriales.Eggerthellaceae.Adlercreutzia,Bacteria.Actinomycetota.Coriobacteriia.Coriobacteriales.Eggerthellaceae.Senegalimassilia,Bacteria.Bacillota.Bacilli.Erysipelotrichales.Erysipelotrichaceae.Holdemanella,Bacteria.Bacillota.Bacilli.Erysipelotrichales.Erysipelotrichaceae.NA,Bacteria.Bacillota.Bacilli.Lactobacillales.Lactobacillaceae.Lactiplantibacillus,Bacteria.Bacillota.Bacilli.Lactobacillales.Lactobacillaceae.Limosilactobacillus,Bacteria.Bacillota.Bacilli.Lactobacillales.Lactobacillaceae.Pediococcus,...,Bacteria.Pseudomonadota.Alphaproteobacteria.Rhodobacterales.Paracoccaceae.Octadecabacter,Bacteria.Pseudomonadota.Alphaproteobacteria.Acetobacterales.Acetobacteraceae.Swingsia,Bacteria.Bacteroidota.Bacteroidia.Flavobacteriales.Flavobacteriaceae.Aurantiacicella,Archaea.Halobacteriota.Halobacteria.Halobacterales.Halobacteriaceae.Halocalculus,pubdate,geo_loc_name,iso,region,tag,value
0,PRJEB11419_ERR5325633,0.0,0.005811,0.005045,0.002162,0.025902,0.0,0.0,0.000000,0.006216,...,0.0,0.0,0.0,0.0,2021-03-17 05:13:05,united arab emirates:unspecified,AE,Northern Africa and Western Asia,acid_reflux,i do not have this condition
4,PRJEB11419_ERR5342221,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,2021-03-17 04:56:09,armenia:unspecified,AM,Northern Africa and Western Asia,acid_reflux,i do not have this condition
14,PRJEB11419_ERR5337878,0.0,0.000000,0.000000,0.000000,0.000119,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,2021-03-16 23:07:17,austria:unspecified,AT,Europe and Northern America,acid_reflux,i do not have this condition
20,PRJEB11419_ERR5337879,0.0,0.000146,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,2021-03-16 23:07:17,austria:unspecified,AT,Europe and Northern America,acid_reflux,i do not have this condition
26,PRJEB11419_ERR5337876,0.0,0.000538,0.000000,0.000269,0.001209,0.0,0.0,0.000179,0.000000,...,0.0,0.0,0.0,0.0,2021-03-16 23:07:17,austria:unspecified,AT,Europe and Northern America,acid_reflux,i do not have this condition
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33808,PRJEB11419_ERR5327905,0.0,0.001256,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,2021-03-17 02:38:28,unspecified,unknown,unknown,acid_reflux,i do not have this condition
33813,PRJEB11419_ERR5327867,0.0,0.000459,0.000121,0.000169,0.001015,0.0,0.0,0.000266,0.000000,...,0.0,0.0,0.0,0.0,2021-03-17 02:38:28,unspecified,unknown,unknown,acid_reflux,"diagnosed by a medical professional (doctor, p..."
33818,PRJEB11419_ERR5328972,0.0,0.000141,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,2021-03-17 09:45:01,unspecified,unknown,unknown,acid_reflux,unspecified
33823,PRJEB11419_ERR5328799,0.0,0.000644,0.000000,0.000000,0.000276,0.0,0.0,0.001105,0.000000,...,0.0,0.0,0.0,0.0,2021-03-17 09:44:59,unspecified,unknown,unknown,acid_reflux,"diagnosed by a medical professional (doctor, p..."


In [47]:
normalized_gi_tags = normalize_multilabel_gi_tags(merged_gi)
merged_gi_normalized = pd.merge(left = merged_gi.drop(columns=['tag','value']), right= normalized_gi_tags, on = 'sample', how = 'left')
#keep disease classes specific
merged_gi_normalized = merged_gi_normalized[merged_gi_normalized.tag != 'GI_other'].dropna()

In [21]:
multilabels = merged_gi_normalized.pivot_table(index = 'sample',columns='tag',values='value', fill_value=0).reset_index()

gi_multilabel_training = (pd.merge(multilabels,merged_gi_normalized.drop_duplicates('sample'),'left','sample')
                          .drop(columns = ['pubdate','geo_loc_name','iso','region','tag','value'])
                          .clean_names()
)
drop_cols = dead_features(gi_multilabel_training)

gi_multilabel_training = gi_multilabel_training.drop(columns = drop_cols)


gi_multilabel_training.to_parquet('../data/processed/gi_multilabel_training.parquet')


### Mental health tags. ended up being too few positives for use in modelling unfortunately :(

In [ ]:
mental_health_merged = filter_by_tag('../data/interim/regional_data/',MENTAL_HEALTH_TAGS)
mental_health_merged.write_parquet('../data/interim/filtered_and_merged/mental_health_microbiomes_merged.parquet')

In [ ]:
#i'm only interested in specific mental illnesses, not general/mixed values. I'm dropping anything vague/general thats mapped to -1
#i want to use this for multilabel classification

tag_mapping = {
    # Core combined
    "depression_bipolar_schizophrenia": -1,

    # General
    "mental_illness": -1,
    "mental_illness_type": -1,

    # Specific disorders
    "mental_illness_type_anorexia_nervosa": "anorexia",
    "mental_illness_type_bipolar_disorder": "bipolar_disorder",
    "mental_illness_type_bulimia_nervosa": "bulimia",
    "mental_illness_type_depression": "depression",
    "mental_illness_type_schizophrenia": "schizophrenia",
    "mental_illness_type_substance_abuse": "substance_abuse",
    "mental_illness_type_unspecified": -1,

    # PTSD variants (normalize both spellings)
    "mental_illness_type_ptsd_posttraumatic_stress_disorder": "ptsd",
    "mental_illness_type_ptsd_post_traumatic_stress_disorder": "ptsd",

    # Keep both depression-related signals
    "has_depression1": "depression",
    "has_depression2": "depression",

    #i wasn't able to locate what scale was being used for these numerical tags, so i'm dropping them
    "depression_index1": -1,
    "depression_index2": -1,

    "anxiety_index1": -1,
    "anxiety_index2": -1,

    "depression_level": -1,
    "depression_status": -1,
    "stress_level": -1,
    "stress_status": -1,
}

value_mapping = {
    'false':0,
    'no':0,
    'not provided':-1,
    'i do not have this condition':0,
    'self-diagnosed':1,
    'not collected':-1,
    'diagnosed by a medical professional (doctor, physician assistant)':1,
    'labcontrol test':1,
    'unspecified':-1
}

def bin_promis_scale(score):
    # https://www.sciencedirect.com/science/article/abs/pii/S0889159119315314?via%3Dihub
    #based on the above paper (where the promis score tag/values originated), under 21 and below is a good threshold for binary binning
    try:
        score =  int(score)
        return 0 if score < 22 else 1
    except (ValueError, TypeError):
        return score
    
mental_health_merged = pd.read_parquet('../data/interim/filtered_and_merged/mental_health_microbiomes_merged.parquet')
mental_health_merged.value = mental_health_merged.value.apply(bin_promis_scale)
mental_health_merged.tag = mental_health_merged.tag.map(tag_mapping)
mental_health_merged.value = mental_health_merged.value.map(value_mapping)
mental_health_merged = (mental_health_merged[~((mental_health_merged.tag == -1) | (mental_health_merged.value == -1))]
                        .dropna(how='any')
                        .reset_index(drop=True))


### BMI

In [2]:
bmi_merged = filter_by_tag('../data/interim/regional_data/',['host_body_mass_index','body_mass_index']).to_pandas()

In [3]:
drop_cols = dead_features(bmi_merged)
bmi_training = (
    bmi_merged
    .drop_duplicates('sample')
    .drop(columns = ['sample','pubdate','geo_loc_name','iso','region','tag']+drop_cols)
    .rename(columns = {'value':'bmi'})
    .dropna(how='any')
    .rename(columns = {col:clean_feature_names(col) for col in bmi_merged.columns})
)
bmi_training.bmi = pd.to_numeric(bmi_training.bmi,errors='coerce')
bmi_training = bmi_training.dropna(how='any').reset_index(drop=True)

#filter to reasonable range of bmis to remove errors from inputting bmi incorrectly
bmi_training = bmi_training[(bmi_training.bmi < 40) & (bmi_training.bmi > 7)].reset_index(drop=True)

bmi_training.to_parquet('../data/processed/bmi_training.parquet')


## PsycGM

'19.71'

In [13]:
sheet_names = ['taxonomy','lineage',
 'diversity','study','study_type_study_level',
 'categories_of_disorder','organism_study_level',''
 'intervention_study_level','sample_study_level',
 'platform']

psycgm_sheets = {}
for num,sheet in enumerate(sheet_names):
    df =  pd.read_excel('../data/raw/psycgm/psycgm_data.xlsx',sheet_name=num)
    psycgm_sheets[sheet] = df

In [20]:
psycgm_sheets['diversity']

,Diversity_ID,Study_ID,Study_type_Diversity_level,Study_type2_Diversity_level,Alpha_diversity_assessment_Diveristy_level,Alpha_diversity_alteration_Diveristy_level,Beta_diversity_assessment_Diveristy_level,Beta_diversity_alteration_Diveristy_level,Comparison_groups_Diversity_level,Intevention_Diversity_level,Organism_Diversity_level_1,Organism_Diversity_level_2,Categories_of_disorder_Diversity_level_1,Categories_of_disorder_Diversity_level_2,Categories_of_disorder_Diversity_level_3,Categories_of_disorder_Diversity_level_4,Sample_Diversity_level,Platform_Diversity_level
0,1,Study 001,Type3,treated vs. untreated healthy group,Observed OTUs; ACE; Chao1; Shannon,Increased,NaN,NaN,lean ground beef diet group vs. standard roden...,Diet and food,CF1 mouse,Mouse,Healthy individuals,Healthy individuals,Healthy individuals,Anxiety,Feces,16S rRNA amplicon sequencing
1,2,Study 002,Type1,psychiatric disorder group vs. control group,ACE; Chao1,Increased,PCA,Separately clustered,severe autism group vs. control group,NaN,Human,Human,Autism spectrum disorder,Autism spectrum disorder,Autism spectrum disorder,Autism spectrum disorder,Feces,16S rRNA amplicon sequencing
2,3,Study 003,Type1,psychiatric disorder group vs. control group,Shannon,NS,NaN,NaN,autistic disorder group vs. control group,NaN,Human,Human,Autistic disorder,Autism spectrum disorder,Autism spectrum disorder,Autism spectrum disorder,Ileal and cecal mucosa,16S rRNA amplicon sequencing
3,4,Study 004,Type1,psychiatric disorder group vs. control group,NaN,NaN,PCA based on the Unifrac analysis,Separately clustered,alcohol group vs. control group,NaN,C57BL/6 mouse,Mouse,Alcoholic liver disease model,Alcoholic liver disease model,Animal models of alcohol-related disorder,Substance-related disorder,Cecal contents,16S rRNA amplicon sequencing
4,5,Study 005,Type1,psychiatric disorder group vs. control group,Faith’s phylogenetic diversity,NS,PCA based on weighted UniFrac distances,Significantly different,grid floor housed group vs. control group,NaN,BALB/c mouse,Mouse,Grid floor housing induced anxiety model with ...,Other animal models of anxiety with depression,Animal models of anxiety with depression,Comorbid anxiety and depression,Cecal contents,16S rRNA amplicon sequencing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,1042,Study 557,Type1,psychiatric disorder group vs. control group,Chao; ACE; Shannon; Simpson; Sobe,NS,NaN,NaN,depression with anxiety group vs. depression w...,NaN,Human,Human,Depression with anxiety symptom,Depression with anxiety symptom,Comorbid anxiety and depression,Comorbid anxiety and depression,Feces,16S rRNA amplicon sequencing
1042,1043,Study 558,Type1,psychiatric disorder group vs. control group,Chao; ACE; Shannon; Simpson; Sobe,NS,NaN,NaN,depression group vs. control group,NaN,Human,Human,Depression,Depression,Depressive disorder,Depression,Feces,16S rRNA amplicon sequencing
1043,1044,Study 559,Type1,psychiatric disorder group vs. control group,ACE; Chao,Decreased,PLS-DA,Separately clustered,alcoholic liver cirrhosis group vs. control group,NaN,Human,Human,Alcoholic liver disease,Alcohol induced disorder,Alcohol-related disorder,Substance-related disorder,Feces,16S rRNA amplicon sequencing
1044,1045,Study 559,Type1,psychiatric disorder group vs. control group,ACE; Chao,Increased,PLS-DA,Separately clustered,alcoholic fatty liver group vs. alcoholic live...,NaN,Human,Human,Alcoholic liver disease,Alcohol induced disorder,Alcohol-related disorder,Substance-related disorder,Feces,16S rRNA amplicon sequencing


## Borenstein